In [1]:
pip install transformers torch transformers_stream_generator tiktoken einops accelerate

  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 12.7 MB/s eta 0:00:00
  Created wheel for transformers_stream_generator: filename=transformers_stream_generator-0.0.5-py3-none-any.whl size=12425 sha256=7283c398b97fa03efb0afba2656a72c2dff355247b7c0812fc5a891289dc7e8e
  Stored in directory: /root/.cache/pip/wheels/95/4a/90/140f7b67d125906f6a165f38aad212ecb4a695ad0d87582437
Successfully built transformers_stream_generator


In [1]:
from transformers import AutoModelForCausalLM, AutoTokenizer, pipeline
from transformers.generation import GenerationConfig
import warnings
warnings.filterwarnings("ignore")
import textwrap
import torch

### stablelm-zephyr-3b

In [ ]:
# Use a pipeline as a high-level helper
from transformers import pipeline

messages = [
    {"role": "user", "content": "Who are you?"},
]
pipe = pipeline("text-generation", model="stabilityai/stablelm-zephyr-3b")
pipe(messages)

In [2]:
# Load model directly
from transformers import AutoTokenizer, AutoModelForCausalLM

tokenizer = AutoTokenizer.from_pretrained("stabilityai/stablelm-zephyr-3b")
model = AutoModelForCausalLM.from_pretrained("stabilityai/stablelm-zephyr-3b")

tokenizer_config.json:   0%|          | 0.00/5.21k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/2.11M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/587 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/599 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/5.59G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/111 [00:00<?, ?B/s]

In [3]:
# Sample customer email text
text = """
Dear WidgetCorp,
I am writing to express my deep disappointment with your service regarding my recent order (Order #98765). I had ordered the "SuperWidget" as a birthday gift for my daughter,
Emily, whose birthday was on March 15th. Despite your website promising delivery within three days, the product arrived only yesterday, missing her birthday entirely.
I attempted to contact your customer service multiple times for an explanation and to expedite the delivery, but to no avail. My calls and emails went unanswered, leaving me
in a difficult position with no gift for Emily’s special day.This experience has left a sour taste, and I am reconsidering future purchases from WidgetCorp. I expect a prompt
explanation and an assurance that such delays will not occur in the future.
Sincerely,
Jane Doe
"""

qa_questions = [
    "Q1: What was the reason for the customer's disappointment with WidgetCorp?",
    "Q2: How did WidgetCorp's customer service respond to the customer's inquiries about the late delivery?"
]

prompt = f"""
Perform the following actions:
1 - Summarize the following text delimited by triple backticks.
2 - Identify the entities in the text delimited by triple backticks.
3 - Answer the following questions delimited by triple backticks based on the text.
4 - Translate the summary into French and back to English.

Separate your answers with line breaks.

Text:
```{text}```
Questions:
```{qa_questions}```
"""

### SUMMARIZATION TASK

In [ ]:
query = "Summarize the following text:"
prompt = query + "\n" + text
inputs = tokenizer(prompt, return_tensors="pt")
# Generate
generate_ids = model.generate(inputs.input_ids, max_new_tokens=100)
print(tokenizer.batch_decode(generate_ids, skip_special_tokens=True, clean_up_tokenization_spaces=False)[0])

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:0 for open-end generation.
The attention mask is not set and cannot be inferred from input because pad token is same as eos token. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.


Summarize the following text:

Dear WidgetCorp,
I am writing to express my deep disappointment with your service regarding my recent order (Order #98765). I had ordered the "SuperWidget" as a birthday gift for my daughter,
Emily, whose birthday was on March 15th. Despite your website promising delivery within three days, the product arrived only yesterday, missing her birthday entirely.
I attempted to contact your customer service multiple times for an explanation and to expedite the delivery, but to no avail. My calls and emails went unanswered, leaving me
in a difficult position with no gift for Emily’s special day.This experience has left a sour taste, and I am reconsidering future purchases from WidgetCorp. I expect a prompt
explanation and an assurance that such delays will not occur in the future.
Sincerely,
Jane Doe

The text describes a customer's disappointment with the delivery of their order from WidgetCorp. The order was for a "SuperWidget" which was intended as a birthday 

### QUESTION ANSWERING TASK

In [ ]:
query = f"""Answer both of the two questions:```{qa_questions}```"""
prompt = query + "\n" + "\n" + text + "\n" + "\n"
inputs = tokenizer(prompt, return_tensors="pt")
output = model.generate(**inputs, do_sample=True, top_p=0.95, top_k=0, max_new_tokens=255)
print(tokenizer.decode(output[0], skip_special_tokens=True))

Setting `pad_token_id` to `eos_token_id`:0 for open-end generation.


Answer both of the two questions:```["Q1: What was the reason for the customer's disappointment with WidgetCorp?", "Q2: How did WidgetCorp's customer service respond to the customer's inquiries about the late delivery?"]```


Dear WidgetCorp,
I am writing to express my deep disappointment with your service regarding my recent order (Order #98765). I had ordered the "SuperWidget" as a birthday gift for my daughter,
Emily, whose birthday was on March 15th. Despite your website promising delivery within three days, the product arrived only yesterday, missing her birthday entirely.
I attempted to contact your customer service multiple times for an explanation and to expedite the delivery, but to no avail. My calls and emails went unanswered, leaving me
in a difficult position with no gift for Emily’s special day.This experience has left a sour taste, and I am reconsidering future purchases from WidgetCorp. I expect a prompt
explanation and an assurance that such delays will not occur in th

### NAME ENTITY RECOGNITION TASK

In [ ]:
query = "Identify the entities and their types (use standard NER terminology) in the following text:"
prompt = query + "\n" + text
inputs = tokenizer(prompt, return_tensors="pt")
output = model.generate(**inputs, do_sample=True, top_p=0.95, top_k=0, max_new_tokens=255)
print(tokenizer.decode(output[0], skip_special_tokens=True))

Setting `pad_token_id` to `eos_token_id`:0 for open-end generation.


Identify the entities and their types (use standard NER terminology) in the following text:

Dear WidgetCorp,
I am writing to express my deep disappointment with your service regarding my recent order (Order #98765). I had ordered the "SuperWidget" as a birthday gift for my daughter,
Emily, whose birthday was on March 15th. Despite your website promising delivery within three days, the product arrived only yesterday, missing her birthday entirely.
I attempted to contact your customer service multiple times for an explanation and to expedite the delivery, but to no avail. My calls and emails went unanswered, leaving me
in a difficult position with no gift for Emily’s special day.This experience has left a sour taste, and I am reconsidering future purchases from WidgetCorp. I expect a prompt
explanation and an assurance that such delays will not occur in the future.
Sincerely,
Jane Doe

Entities and their types in the text:

1. WidgetCorp - (Organization) - Type: For-profit company
2. Or

### TRANSLATION (ENGLISH TO FRENCH TO ENGLISH) TASK

In [4]:
query = "Translate the text into French language and then translate the French text generated back to English. Give both translations results."
prompt = query + "\n" + "\n" + text + "\n" + "\n"
inputs = tokenizer(prompt, return_tensors="pt")
output = model.generate(**inputs, do_sample=True, top_p=0.95, max_new_tokens=300)
print(tokenizer.decode(output[0], skip_special_tokens=True))

Setting `pad_token_id` to `eos_token_id`:0 for open-end generation.


Translate the text into French language and then translate the French text generated back to English. Give both translations results.


Dear WidgetCorp,
I am writing to express my deep disappointment with your service regarding my recent order (Order #98765). I had ordered the "SuperWidget" as a birthday gift for my daughter,
Emily, whose birthday was on March 15th. Despite your website promising delivery within three days, the product arrived only yesterday, missing her birthday entirely.
I attempted to contact your customer service multiple times for an explanation and to expedite the delivery, but to no avail. My calls and emails went unanswered, leaving me
in a difficult position with no gift for Emily’s special day.This experience has left a sour taste, and I am reconsidering future purchases from WidgetCorp. I expect a prompt
explanation and an assurance that such delays will not occur in the future.
Sincerely,
Jane Doe



Let's traduire le texte en français :

Cher WidgetCorp,
J